In [ ]:
import pandas as pd

max_items_to_transfer = 1

works_metadata = pd.read_csv('../works_multiprop.csv', na_filter=False, dtype = str)
existing_images = pd.read_csv('commons_images.csv', na_filter=False, dtype = str) # Don't make the Q IDs the index!

# Step through the list of files with uploaded images to check if their data need to be transferred to works metadata
items_transferred = 0
for index, work in existing_images.iterrows():
    
    # If the work failed to upload, skip it.
    if work['commons_id'] == 'fail':
        continue
        
    # Skip if the item doesn't have an image name.
    if work['image_name'] == '':
        continue
        
    # Find out if the image is already in the works metadata
    qid = work['qid']
    # Search in the qid column of the works metadata to find rows that match the current item qid
    row_series = works_metadata[works_metadata['qid'].str.contains(qid)]
    if len(row_series) == 0:
        print('uploaded image Wikidata record for', qid, 'not found in works metadata.')
        continue # skip to next uploaded image
    elif len(row_series) >= 2:
        print('more than Wikidata record found for', qid, 'in works metadata.')
        continue # skip to next uploaded image
    else:
        # The result of getting the image_uuid item is itself a series, so its
        # Numpy array has to be accessed in order to address its first item 0.
        #print(row_series['image_uuid'].array[0])
        if row_series['image_uuid'].array[0] != '': # If image is already uploaded, it will have a UUID
            continue # skip to the next image
        else:
            # Series have only one dimension, so the value returned for the .index method has only one (0th) item.
            row_index = row_series.index[0]

    # Find the image name and insert it into the works metadata DataFrame
    image_name = work['image_name']
    print('transferring:', image_name)
    works_metadata.at[row_index, 'image'] = image_name

    items_transferred += 1
    if items_transferred >= max_items_to_transfer:
        break

# Write the updated dataframe to CSV
works_metadata.to_csv('../works_multiprop.csv', index = False)
print('done')